In [ ]:
import basic
import geopandas as gpd
import os, shutil
import numpy as np

In [ ]:
ml = basic.load_model()

In [ ]:
ghb = ml.ghb

sp = ghb.stress_period_data[0]

ghb.stress_period_data = sp
sp['cond'] = .0005

ghb.write_file()

In [ ]:
bas = ml.bas6.ibound.array

In [ ]:
bas = bas==-1

In [ ]:
k, i, j = np.indices(bas.shape)

In [ ]:
import flopy
from flopy.modflow import ModflowGhb

# ModflowGhb()


In [ ]:
head, hdsobj = basic.get_heads(ml)

In [ ]:
bas = bas.reshape((-1,1))
head = head.reshape((-1,1))
k, i, j = k.reshape((-1,1)), i.reshape((-1,1)), j.reshape((-1,1))

In [ ]:
cond = np.ones(head.shape)*.1

In [ ]:
stress_period_data = ModflowGhb.get_empty(ncells= bas[bas].shape[0])

In [ ]:
stress_period_data['k'] = k[bas]
stress_period_data['i'] = i[bas]
stress_period_data['j'] = j[bas]
stress_period_data['bhead'] = head[bas]
stress_period_data['cond'] = cond[bas]
stress_period_data

In [ ]:
flopy.modflow.ModflowWel.get_empty()

In [ ]:
ghb = ModflowGhb(ml, ipakcb=1,stress_period_data=stress_period_data, filenames = 'RR.ghb')

In [ ]:
ghb.write_file()

In [ ]:
ghb.plot(per = 0)

In [ ]:
import conda_scripts.arich_functions as af
import conda_scripts.utils.folium_maps as fm
import pandas as pd

In [ ]:
grid = gpd.read_file('GIS/grid.shp')

In [ ]:
grid.head()

In [ ]:
modelgrid = af.get_model_shp(ml.modelgrid)

In [ ]:
modelgrid.head()

In [ ]:
stress_period_data = ghb.stress_period_data[0]

In [ ]:
gh = pd.DataFrame(stress_period_data)
gh.loc[:,'row'] = gh.loc[:,'i']+1
gh.loc[:,'column'] = gh.loc[:,'j']+1
gh = pd.merge(gh, grid.drop(columns = 'geometry'), on = ['row','column'])

gh = gpd.GeoDataFrame(pd.merge(gh,modelgrid, on = ['i','j']), crs = 2226)

# 

m  = gh.query("k==0").explore('cond', name = 'layer 1')
gh.query("k==1").explore('cond', name = 'layer 2', m = m)
grid.query("botm_1!=999").dissolve().explore(m = m, name = 'grid')
fm.add_layers(m)
m

In [ ]:
import Hydrographs

In [ ]:
datestart = '4/16/2016'

In [ ]:
obs =  Hydrographs.load_obs('MW93_14.csv', datestart,numdays=109)

In [ ]:
obs.plot()

In [ ]:
import conda_scripts.gwplot_fancy as gwp
import matplotlib.pyplot as plt

In [ ]:
skip_gw_data = True

In [ ]:
nwp = gwp.fancy_plot('LRR0061',group = None,
                     filename='test.png',
                     allinfo=None,
                     do_regress=False)

nwp.do_plot(False, skip_gw_data=skip_gw_data,
            map_buffer_size = 2500, seasonal = False,
            plot_dry = False, plot_wet = False,
            maptype = 'ctx.USGS.USTopo')

hds, hdsobj = basic.get_heads(ml)
idx = (0, 140, 66)
head = Hydrographs.get_ts(idx, hdsobj, datestart)
head.plot(ax = nwp.upleft)


obs.rename(columns = {'Value':'Observed'}).plot(ax = nwp.upleft)

nwp.fig

In [ ]:
nwp.fig

In [ ]:
plt.savefig('test.png')

In [ ]:
head.plot()

In [ ]:
grid.query("botm_1!=999").explore( name = 'grid')